In [1]:
import numpy as np 
import cv2 
import sklearn  
import pickle

In [2]:
%cd C:\Users\welcome\Downloads\Face Recognition

C:\Users\welcome\Downloads\Face Recognition


In [3]:
#face detection 
face_detector_model=cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt','./models/res10_300x300_ssd_iter_140000_fp16.caffemodel') 

#feature extraction
face_feature_model=cv2.dnn.readNetFromTorch('./models/openface.nn4.small2.v1.t7')

#face recognition 
face_recognition_model=pickle.load(open('./models/machinelearning_face_person_identity.pkl',mode='rb'))

In [4]:
#emotion recognition 
emotion_recognition_model=pickle.load(open('./models/machinelearning_face_emotion_identity.pkl',mode='rb'))

In [12]:
def pipeline_model(path):
    img=cv2.imread(path)
    image=img.copy()
    h,w=img.shape[:2]

    #face detection 
    img_blob=cv2.dnn.blobFromImage(img,1,(300,300),(104,177,123),swapRB=False,crop=False)
    face_detector_model.setInput(img_blob)
    detections=face_detector_model.forward()
    
    
    #machinelearning results 
    machinelearningresults=dict(face_detect_score=[],face_name=[],face_name_score=[],emotion_name=[],count=[]) 
    
    
    
    count=1


    if len(detections)>0:
        for i, confidence in enumerate(detections[0,0,:,2]):
            if confidence>0.5:
                box=detections[0,0,i,3:7]*np.array([w,h,w,h])
                startx,starty,endx,endy=box.astype(int)

                cv2.rectangle(image,(startx,starty),(endx,endy),(0,255,0))

                #feature extraction 
                face_roi=img[starty:endy,startx:endx]
                face_blob=cv2.dnn.blobFromImage(face_roi,1/255,(96,96),(0,0,0),swapRB=True,crop=True)
                face_feature_model.setInput(face_blob)
                vectors=face_feature_model.forward()

                #predict with machine learning for face recognition
                face_name=face_recognition_model.predict(vectors)[0]
                face_score=face_recognition_model.predict_proba(vectors).max() 

                #predict with machine learning for emotion recognition 
                emotion_name=emotion_recognition_model.predict(vectors)[0]


                text_face='{} : {:.0f} %'.format(face_name,100*face_score)
                text_emotion='{}'.format(emotion_name)
                cv2.putText(image,text_face,(startx,starty),cv2.FONT_HERSHEY_PLAIN,1,(255,255,255),2)
                cv2.putText(image,text_emotion,(startx,endy),cv2.FONT_HERSHEY_PLAIN,1,(255,255,255),2)
                
                
                machinelearningresults['count'].append(count) 
                machinelearningresults['face_detect_score'].append(confidence)
                machinelearningresults['face_name'].append(face_name)
                machinelearningresults['face_name_score'].append(face_score)
                machinelearningresults['emotion_name'].append(emotion_name)
                
                
                count+=1
                
    return machinelearningresults


In [13]:
results=pipeline_model('./data/test/president.jpg')  
print(results)


{'face_detect_score': [0.9999292], 'face_name': ['Barack Obama'], 'face_name_score': [0.6011176631454145], 'emotion_name': ['happy'], 'count': [1]}
